# Install

In [1]:
# Libraries you might not have
# !python3 -m pip install --upgrade nbconvert 
# !python3 -m pip install --upgrade nbstripout 
# !python3 -m pip install tomotopy
# !python3 -m pip install sklearn

# Imports

In [2]:
import nltk
# Things to install from nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/dylfox21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dylfox21/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
%load_ext autoreload
%autoreload 2

import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict

import dataloader
import bow
import slda
import post_classifier
import aggregate
import user_classifier

# Process Train Data

In [4]:
# Uncomment below to process data if you have not already

# POSTPATH = './Data/crowd/train/shared_task_posts.csv'
# LABELPATH = './Data/crowd/train/crowd_train.csv'
# USERPATH = './Data/crowd/train/task_C_train.posts.csv'

# users = dataloader.load_user_subset_from_train(USERPATH, subset = 1000)
    
# user_to_post, post_to_words, post_to_metadata = dataloader.load_posts(POSTPATH, user_subset = users, append_title = True)
# post_to_label = dataloader.load_classification(LABELPATH, user_to_post, post_to_words, post_to_metadata, user_subset = users)
# filtered_data, sw_posts, sw_timestamps = dataloader.filter_posts(post_to_label, post_to_metadata)
# print(len(filtered_data))
# filtered_data = dataloader.filter_near_SW(filtered_data,post_to_metadata, sw_timestamps)
# print(len(filtered_data))

# filtered_data = dataloader.filter_stopwords(filtered_data)
# sw_posts = dataloader.filter_stopwords(sw_posts)

In [5]:
# FOLDERPATH = './Processing/crowd_processed/'
# dataloader.save_to_folder(FOLDERPATH, user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps)

# Load Process Train Data

In [6]:
FOLDERPATH = './Processing/crowd_processed/'
user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = dataloader.load_from_folder(FOLDERPATH)

In [7]:
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'a']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'b']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'c']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'd']))
print(len(filtered_data))

878
203
632
2025
29200


# SLDA Model: Train

## Train Topic Model

In [8]:
model = slda.train_slda_model_from_data(filtered_data, topics=40)

100%|██████████| 29200/29200 [00:00<00:00, 104417.22it/s]


Beginning sLDA training...
Iteration: 0	Log-likelihood: -10.114199139920588
Iteration: 100	Log-likelihood: -9.148173977060692
Iteration: 200	Log-likelihood: -9.036787163578026
Iteration: 300	Log-likelihood: -9.002368434538326
Iteration: 400	Log-likelihood: -8.979918123882936
Iteration: 500	Log-likelihood: -8.967662217962195
Iteration: 600	Log-likelihood: -8.957691087762342
Iteration: 700	Log-likelihood: -8.94868466773021
Iteration: 800	Log-likelihood: -8.938877119845548
Iteration: 900	Log-likelihood: -8.930469770871628
Finished Training


## Topic Model Outputs

In [9]:
slda_coefficients = model.get_regression_coef(0)
data = []
for k in range(model.k):
    top_words = model.get_topic_words(k, top_n=40)
    words = [word for (word, float) in top_words]
    words = ", ".join(words)
    data.append([words, slda_coefficients[k]])
    
indices = np.array(slda_coefficients).argsort()
data = np.array(data)
data = data[indices]

pd.DataFrame(data, columns=["Topic", "Suicidality Coefficient"])

,Topic,Suicidality Coefficient
0,"person, big, babes, girl, sexy, f, teen, hot, ...",-5.741025924682617
1,"order, mail, person, seeds, biodynamic, organi...",-5.261110782623291
2,"url, us, min, survey, gt95, person, short, stu...",-5.080322265625
3,"person, music, song, video, rock, songs, love,...",-4.970279693603516
4,"’, person, “, ”, –, cannabis, marijuana, says,...",-4.694401741027832
5,"person, persons, vs, 2, show, 1, season, 3, te...",-4.436217308044434
6,"person, car, oc, bike, new, dog, x, city, foun...",-4.012282848358154
7,"gt, person, xb1, psn, ps4, vog, invite, x1, lv...",-3.821331739425659
8,"person, trail, gun, completed, treasure, elite...",-3.801435708999634
9,"person, keys, w, h, deck, cards, real, porn, u...",-3.7838666439056396


In [10]:
# Uncomment to print example of overly negative topic
# print(data[39][0])

## sLDA Features: Train

In [ ]:
vector_train = slda.get_topic_vecs(model, filtered_data)

  0%|          | 3/29200 [00:00<20:57, 23.22it/s]

Getting topic distributions...


 83%|████████▎ | 24340/29200 [14:38<02:58, 27.27it/s]

In [ ]:
#Uncomment to print example feature vector
print(vector_train['hw4uh'][0])

# BOW

In [ ]:
word2index,index2word = bow.generate_vocabulary(filtered_data)
vector_train = bow.get_PCA_vectors_from_post_set(filtered_data, word2index)

# Post Classifier: Train

## Post Classifier: Train

In [18]:
X_train = np.array([ vector_train[key][0] for key in vector_train.keys()])
y_train = np.array([ vector_train[key][1] for key in vector_train.keys()])
y_train = y_train.reshape(np.shape(y_train)[0])

In [19]:
print(np.shape(X_train))
print(np.shape(y_train))

(29200, 40)
(29200,)


### Logistic Regression

In [86]:
#UNCOMMENT TO RUN GRID SEARCH CV
#p_clf = post_classifier.PostClassification("LogReg")
#param_dict = {'C':[0.2,0.5,0.7,1,1.5,2,5]}
#p_clf.train_grid_search_CV(X_train, y_train, param_dict, groups=5)

#RUN WITH OPTIMAL PARAMETERS
p_clf = post_classifier.PostClassification("LogReg")
p_clf.train(X_train, y_train)


### Linear SVM

In [ ]:
#UNCOMMENT TO RUN GRID SEARCH CV
#p_clf = post_classifier.PostClassification("LinearSVM")
#param_dict = {'C':[0.2,0.5,1,2]}
#p_clf.train_grid_search_CV(X_train, y_train, param_dict, groups=5)

p_clf = post_classifier.PostClassification("LinearSVM")
p_clf.train(X_train, y_train)

### RBF SVM

In [89]:
#UNCOMMENT TO RUN GRID SEARCH CV
#p_clf = post_classifier.PostClassification("RbfSVM")
#param_dict = {'C':[0.5,1,2,5]}
#p_clf.train_grid_search_CV(X_train, y_train, param_dict, groups=5)

p_clf = post_classifier.PostClassification("RbfSVM")
p_clf.train(X_train, y_train)

### AdaBoost

In [ ]:
p_clf = post_classifier.PostClassification("AdaBoost")
p_clf.train(X_train, y_train)

### Random Forest

In [ ]:
p_clf = post_classifier.PostClassification("RandomForest")
p_clf.train(X_train, y_train)

### Multi-layer Perceptron

In [ ]:
#UNCOMMENT TO RUN GRID SEARCH CV
#p_clf = post_classifier.PostClassification("MLP")
#param_dict = {'hidden_layer_sizes':[(64,64),(64,64,64),(32,32), (32,32,32)], 'learning_rate': ('constant', 'adaptive')}
#p_clf.train_grid_search_CV(X_train, y_train, param_dict, groups=5)

p_clf = post_classifier.PostClassification("MLP")
p_clf.train(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] hidden_layer_sizes=(64, 64), learning_rate=constant .............
Iteration 1, loss = 0.36824695


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Iteration 2, loss = 0.22893707
Iteration 3, loss = 0.22040853
Iteration 4, loss = 0.21889144
Iteration 5, loss = 0.21825781
Iteration 6, loss = 0.21825892
Iteration 7, loss = 0.21758994
Iteration 8, loss = 0.21739908
Iteration 9, loss = 0.21706191
Iteration 10, loss = 0.21659998
Iteration 11, loss = 0.21629070
Iteration 12, loss = 0.21598504
Iteration 13, loss = 0.21545088
Iteration 14, loss = 0.21510306
Iteration 15, loss = 0.21480809
Iteration 16, loss = 0.21439595
Iteration 17, loss = 0.21383275
Iteration 18, loss = 0.21339007
Iteration 19, loss = 0.21319979
Iteration 20, loss = 0.21244893
Iteration 21, loss = 0.21198069
Iteration 22, loss = 0.21164398
Iteration 23, loss = 0.21109092
Iteration 24, loss = 0.21090719
Iteration 25, loss = 0.21012951
Iteration 26, loss = 0.20956350
Iteration 27, loss = 0.20917865
Iteration 28, loss = 0.20868152
Iteration 29, loss = 0.20818804
Iteration 30, loss = 0.20780216
Iteration 31, loss = 0.20696069
Iteration 32, loss = 0.20665677
Iteration 33, lo

Iteration 254, loss = 0.15880114
Iteration 255, loss = 0.15880401
Iteration 256, loss = 0.15840758
Iteration 257, loss = 0.15829830
Iteration 258, loss = 0.15799726
Iteration 259, loss = 0.15826974
Iteration 260, loss = 0.15793667
Iteration 261, loss = 0.15895956
Iteration 262, loss = 0.15737259
Iteration 263, loss = 0.15746922
Iteration 264, loss = 0.15829458
Iteration 265, loss = 0.15798226
Iteration 266, loss = 0.15730954
Iteration 267, loss = 0.15807062
Iteration 268, loss = 0.15809292
Iteration 269, loss = 0.15716662
Iteration 270, loss = 0.15695934
Iteration 271, loss = 0.15797640
Iteration 272, loss = 0.15690208
Iteration 273, loss = 0.15669954
Iteration 274, loss = 0.15737728
Iteration 275, loss = 0.15582152
Iteration 276, loss = 0.15625128
Iteration 277, loss = 0.15753511
Iteration 278, loss = 0.15706201
Iteration 279, loss = 0.15560097
Iteration 280, loss = 0.15676705
Iteration 281, loss = 0.15583503
Iteration 282, loss = 0.15562944
Iteration 283, loss = 0.15515345
Iteration 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


...

/home/dylfox21/miniconda2/envs/cmsc773/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV]  hidden_layer_sizes=(64, 64, 64), learning_rate=constant, total=   4.5s
[CV] hidden_layer_sizes=(64, 64, 64), learning_rate=constant .........
Iteration 1, loss = 0.27944298
Iteration 2, loss = 0.22545159
Iteration 3, loss = 0.22152250
Iteration 4, loss = 0.22060780
Iteration 5, loss = 0.22005381
Iteration 6, loss = 0.21962768
Iteration 7, loss = 0.21924166
Iteration 8, loss = 0.21889472
Iteration 9, loss = 0.21867534
Iteration 10, loss = 0.21794870
Iteration 11, loss = 0.21771808
Iteration 12, loss = 0.21697924
Iteration 13, loss = 0.21665664
Iteration 14, loss = 0.21631434
Iteration 15, loss = 0.21596737
Iteration 16, loss = 0.21521972
Iteration 17, loss = 0.21442772
Iteration 18, loss = 0.21403365
Iteration 19, loss = 0.21377381
Iteration 20, loss = 0.21292027
Iteration 21, loss = 0.21229092
Iteration 22, loss = 0.21171847
Iteration 23, loss = 0.21101377
Iteration 24, loss = 0.21061803
Iteration 25, loss = 0.21033032
Iteration 26, loss = 0.20961726
Iteration 27, loss = 0.209285

Iteration 249, loss = 0.14160223
Iteration 250, loss = 0.14132450
Iteration 251, loss = 0.14104346
Iteration 252, loss = 0.13989801
Iteration 253, loss = 0.14169607
Iteration 254, loss = 0.14157201
Iteration 255, loss = 0.14085691
Iteration 256, loss = 0.13971462
Iteration 257, loss = 0.13939176
Iteration 258, loss = 0.13972223
Iteration 259, loss = 0.14077287
Iteration 260, loss = 0.13990293
Iteration 261, loss = 0.13926337
Iteration 262, loss = 0.13856078
Iteration 263, loss = 0.13873035
Iteration 264, loss = 0.14272488
Iteration 265, loss = 0.14078976
Iteration 266, loss = 0.14030420
Iteration 267, loss = 0.13986931
Iteration 268, loss = 0.14085776
Iteration 269, loss = 0.13748969
Iteration 270, loss = 0.13822947
Iteration 271, loss = 0.13686530
Iteration 272, loss = 0.13616575
Iteration 273, loss = 0.13784127
Iteration 274, loss = 0.13818781
Iteration 275, loss = 0.13692580
Iteration 276, loss = 0.13685067
Iteration 277, loss = 0.13972064
Iteration 278, loss = 0.13688000
Iteration 

/home/dylfox21/miniconda2/envs/cmsc773/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV]  hidden_layer_sizes=(64, 64, 64), learning_rate=adaptive, total=   6.9s
[CV] hidden_layer_sizes=(64, 64, 64), learning_rate=adaptive .........
Iteration 1, loss = 0.35481696
Iteration 2, loss = 0.22936476
Iteration 3, loss = 0.22399871
Iteration 4, loss = 0.22337091
Iteration 5, loss = 0.22261523
Iteration 6, loss = 0.22224596
Iteration 7, loss = 0.22158103
Iteration 8, loss = 0.22150451
Iteration 9, loss = 0.22119142
Iteration 10, loss = 0.22037666
Iteration 11, loss = 0.22010300
Iteration 12, loss = 0.21940108
Iteration 13, loss = 0.21904189
Iteration 14, loss = 0.21872194
Iteration 15, loss = 0.21795867
Iteration 16, loss = 0.21805599
Iteration 17, loss = 0.21716263
Iteration 18, loss = 0.21633298
Iteration 19, loss = 0.21659343
Iteration 20, loss = 0.21549661
Iteration 21, loss = 0.21474561
Iteration 22, loss = 0.21492138
Iteration 23, loss = 0.21396217
Iteration 24, loss = 0.21307799
Iteration 25, loss = 0.21373568
Iteration 26, loss = 0.21220239
Iteration 27, loss = 0.211701

## Predict Post Classifier: Train

In [90]:
y_pred_train = p_clf.test(X_train)

In [91]:
p_clf.get_metrics(y_train, y_pred_train)

{'accuracy': 0.7092808219178082,
 'precision': 0.16769483857701006,
 'recall': 0.8054320987654321,
 'f1': 0.27759339630669727}

In [92]:
print(sum(y_pred_train))
print(sum(y_train))

9726
2025


# User Classfier: Train

## Aggregate: Train

In [93]:
# change y from a, b, c, d, control to -, 1
user_to_y_train = defaultdict(int)
for data in tqdm.tqdm(filtered_data.keys()):
    user_to_y_train[filtered_data[data][0]] = (1 if filtered_data[data][2] == 'd' else 0)

100%|██████████| 29200/29200 [00:00<00:00, 1048459.31it/s]


In [94]:
post_to_uypred_train = defaultdict(list)

for i, post_id in enumerate(vector_train.keys()):
    user_id = filtered_data[post_id][0]
    post_to_uypred_train[post_id] = [user_id, y_pred_train[i]]

In [95]:
user_to_post_label_train = aggregate.aggregate_posts(FOLDERPATH, post_to_uypred_train)

Saved aggreagation of user to post labels...


## Argmax: Train

In [96]:
u_clf_train = user_classifier.UserClassification(user_to_post_label_train)
user_to_ypred_train = u_clf_train.argmax()

In [97]:
user_y_train = []
user_y_pred_train = []
for user_id in user_to_ypred_train:
    user_y_train.append(user_to_y_train[user_id])
    user_y_pred_train.append(user_to_ypred_train[user_id])

In [98]:
u_clf_train.get_metrics(user_y_train, user_y_pred_train)

{'accuracy': 0.7214452214452215,
 'precision': 0.3967828418230563,
 'recall': 0.9135802469135802,
 'f1': 0.5532710280373832}

# Process Data: Test

In [35]:
POSTPATH2 = './Data/crowd/test/shared_task_posts_test.csv'
LABELPATH2 = './Data/crowd/test/crowd_test_C.csv'
USERPATH2 = './Data/crowd/test/task_C_test.posts.csv'
    
user_to_post_test, post_to_words_test, post_to_metadata_test = dataloader.load_posts(POSTPATH2, append_title = True)
post_to_label_test = dataloader.load_classification(LABELPATH2, user_to_post_test, post_to_words_test, post_to_metadata_test)
filtered_data_test, sw_posts_test, sw_timestamps_test = dataloader.filter_posts(post_to_label_test, post_to_metadata_test)
print(len(filtered_data_test))
filtered_data_test = dataloader.filter_near_SW(filtered_data_test, post_to_metadata_test, sw_timestamps_test)
print(len(filtered_data_test))

filtered_data_test = dataloader.filter_stopwords(filtered_data_test)
sw_posts_test = dataloader.filter_stopwords(sw_posts_test)

 13%|█▎        | 1910/14447 [00:00<00:01, 7915.41it/s]

Tokenizing sentences...


  1%|          | 130/14447 [00:00<00:11, 1290.87it/s]

Normalizing...
Tokenizing sentences into words...


100%|██████████| 14056/14056 [00:00<00:00, 946112.96it/s]

249
14056
Filtering posts far away from SW posts...
5637


In [36]:
FOLDERPATH2 = './Processing/crowd_processed_test/'
dataloader.save_to_folder(FOLDERPATH2, user_to_post_test, post_to_metadata_test, filtered_data_test, sw_posts_test, sw_timestamps_test)

# Load Process Data: Test

In [37]:
FOLDERPATH2 = './Processing/crowd_processed_test/'
user_to_post_test, post_to_metadata_test, filtered_data_test, sw_posts_test, sw_timestamps_test = dataloader.load_from_folder(FOLDERPATH2)

In [38]:
print(len([filtered_data_test[key] for key in filtered_data_test.keys() if filtered_data_test[key][2] == 'a']))
print(len([filtered_data_test[key] for key in filtered_data_test.keys() if filtered_data_test[key][2] == 'b']))
print(len([filtered_data_test[key] for key in filtered_data_test.keys() if filtered_data_test[key][2] == 'c']))
print(len([filtered_data_test[key] for key in filtered_data_test.keys() if filtered_data_test[key][2] == 'd']))
print(len(filtered_data_test))

4922
58
257
400
5637


# Feature Extraction: Test

## sLDA

In [39]:
X_test, y_test = slda.vectorize_data_set(model, FOLDERPATH2)

  0%|          | 3/5637 [00:00<03:31, 26.65it/s]

Getting topic distributions...


100%|██████████| 5637/5637 [03:13<00:00, 29.06it/s]


In [40]:
vector_test = slda.get_topic_vecs(model, filtered_data_test)

  0%|          | 3/5637 [00:00<04:13, 22.19it/s]

Getting topic distributions...


100%|██████████| 5637/5637 [03:12<00:00, 29.22it/s]


# Post Classifier: Test

## Predict Post Classifier: Test

In [99]:
y_pred_test = p_clf.test(X_test)

In [100]:
p_clf.get_metrics(y_test, y_pred_test)

{'accuracy': 0.5887883626042221,
 'precision': 0.09260832625318606,
 'recall': 0.545,
 'f1': 0.15831517792302105}

In [101]:
print(sum(y_pred_test))
print(sum(y_test))

2354
[400]


# User Classifier: Test

## Aggregate: Test

In [102]:
# change y from a, b, c, d, control to -, 1
user_to_y_test = defaultdict(int)
for data in tqdm.tqdm(filtered_data_test.keys()):
    user_to_y_test[filtered_data_test[data][0]] = (1 if filtered_data_test[data][2] == 'd' else 0)

100%|██████████| 5637/5637 [00:00<00:00, 1238776.68it/s]


In [103]:
len(filtered_data_test)

5637

In [104]:
len(vector_test)

5637

In [105]:
post_to_uypred_test = defaultdict(list)

for i, post_id in enumerate(vector_test.keys()):
    user_id = filtered_data_test[post_id][0]
    post_to_uypred_test[post_id] = [user_id, y_pred_test[i]]

In [106]:
user_to_post_label_test = aggregate.aggregate_posts(FOLDERPATH2, post_to_uypred_test)

Saved aggreagation of user to post labels...


## Argmax: Test

In [107]:
u_clf_test = user_classifier.UserClassification(user_to_post_label_test)
user_to_ypred_test = u_clf_test.argmax()

In [108]:
user_y_test = []
user_y_pred_test = []
for user_id in user_to_ypred_test:
    user_y_test.append(user_to_y_test[user_id])
    user_y_pred_test.append(user_to_ypred_test[user_id])

In [109]:
u_clf_test.get_metrics(user_y_test, user_y_pred_test)

{'accuracy': 0.5981308411214953,
 'precision': 0.24742268041237114,
 'recall': 0.6486486486486487,
 'f1': 0.3582089552238806}

## Threshod: Test

In [ ]:
u_clf_test = user_classifier.UserClassification(user_to_post_label_test)

In [ ]:
u_clf_test.find_threshold(user_to_y_test)

In [ ]:
user_to_ypred_test = u_clf_test.minimum(1)

In [ ]:
user_y_test = []
user_y_pred_test = []
for user_id in user_to_ypred_test:
    user_y_test.append(user_to_y_test[user_id])
    user_y_pred_test.append(user_to_ypred_test[user_id])

In [ ]:
u_clf_test.get_metrics(user_y_test, user_y_pred_test)